## 0.0 Pacotes e Bibliotecas

In [21]:
# Manipulação de Dados
import pandas as pd
import numpy  as np

import pickle as pkl

## 1.0 Carregamento dos Dados

In [22]:
df1_fato = pd.read_csv('../BaseDados/DadosTreino.csv', delimiter=',', low_memory=False)
df1_dimensao = pd.read_csv('../BaseDados/DadosLoja.csv', delimiter=',', low_memory=False)
df1 = df1_fato.merge(df1_dimensao, how='left', on='Store')

### 1.1 Seleção dos Dados

Como o nosso objetivo é realizar a previsão dos próximos 6 dias de vendas das lojas, podemos, após um entendimento das variáveis, verificar quais delas fazem sentido. 

1. Ao analisar, temos que duas variávies não fazem sentido para nós. São elas: 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'.
2. Obtendo apenas os dados dos últimos 7 dias de vendas.

Portanto, vamos remover essas variáveis do nosso dataset e aplicar os filtros, de modo que, iniciemos nosso trabalho.

In [23]:
df1 = df1[df1['Open'] == 1].drop(columns=['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1).copy()

## 2.0 Engenharia de Atributos

O objetivo dessa etapa é conseguir ver a possibilidade de informações que estão 'escondidas' nos dados. Podemos obervar que o nosso dataset é bem limitado, sendo assim, vamos inicialmente construir nossa variável que é as vendas dos próximos 6 dias.

In [18]:
# Cópia dos Dados
df2 = df1.copy()


# Tratamento de Data
# Ajuste da Variável Data
df2['Date'] = pd.to_datetime(df2['Date'], format='%Y-%m-%d')

# Ordenamento dos Dados
df2_sort = df2.sort_values(by=['Store', 'Date'], ascending=True).reset_index(drop=True).copy()

max_data = df2_sort.groupby('Store')['Date'].transform('max')
df2_sort = df2_sort[df2_sort['Date'] >= (max_data - pd.Timedelta(days=6))]

Observe que a variável stateHolyday é uma variável categórica, vamos criar variáveis dummies de modo trazer essas variáveis.

In [19]:
# Gerando as variáveis dummies para os feriados
df2_sort = (pd.get_dummies(data=df2_sort, columns=['StateHoliday'], dtype=np.int64)
              .drop(columns=['StateHoliday_0', 'StateHoliday_b', 'StateHoliday_c'], axis=1)
              .rename(columns={'StateHoliday_a':'PublicHoliday', 'StateHoliday_b':'EasterHoliday', 'StateHoliday_c':'Christmas_Holiday'}))

KeyError: "['StateHoliday_b', 'StateHoliday_c'] not found in axis"

In [6]:
# Vendas do último dia
df2_sort['SalesLastDay'] = df2_sort.groupby('Store')['Sales'].shift(1)

# Média dos últimos 7 dias
df2_sort['MeanSalesLastSevenDays'] = df2_sort.groupby('Store')['Sales'].shift(1).rolling(7).mean()

# Total de vendas dos últimos 7 dias
df2_sort['TotalSalesLastSevenDays'] = df2_sort.groupby('Store')['Sales'].shift(1).rolling(7).sum()

# Total de Promoções aplicadas nos últimos 7 dias
df2_sort['TotalPromoLastSevenDays'] = df2_sort.groupby('Store')['Promo'].shift(1).rolling(7).sum()

# Total de Feriados Públicos nos últimos 7 dias
df2_sort['TotalPublicHolidaysLastSevenDays'] = df2_sort.groupby('Store')['PublicHoliday'].shift(1).rolling(7).sum()

# Total de promoções 2 ativadas nos últimos 7 dias
df2_sort['TotalPromo2LastSevenDays'] = df2_sort.groupby('Store')['Promo2'].shift(1).rolling(7).sum()

# Total de feriados escolares no últimos 7 dias
df2_sort['TotalSchoolHolidayLastSevenDays'] = df2_sort.groupby('Store')['SchoolHoliday'].shift(1).rolling(7).sum()

# Anos de Competição
df2_sort['YearsCompetition'] = df2_sort['Date'].max().year - df2_sort['CompetitionOpenSinceYear']

Aqui vamos selecionar apenas as variáveis que façam sentido para o nosso problema. 

In [7]:
df2_sort = df2_sort[['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'YearsCompetition', 'Sales', 'Customers', 'Promo','Promo2', 'PublicHoliday', 'SalesLastDay',
                     'MeanSalesLastSevenDays', 'TotalSalesLastSevenDays', 'TotalPromoLastSevenDays', 'TotalPromo2LastSevenDays', 'TotalSchoolHolidayLastSevenDays', 'TotalPublicHolidaysLastSevenDays']]

# Renomeando colunas
df2_sort = df2_sort.rename(columns={'Sales':'SalesDay', 'Customers': 'CustomersDay', 'PromoDay':'Promo2Day', 'PublicHoliday':'PublicHolidayDay'})

Observe que a nossa variável alvo nossa é a TotalSalesNextSixDays. Sendo assim, nossa variável alvo não pode ser nula. Além disso, de modo que tenhamos o máximo de informação consolidada, vamos eliminar a quantidade as variáveis nulas em relação a soma móveis, pois pelo fato de ser nulo, quer dizer que não há informações disponíveis, logo, não está trazendo o conteúdo que precisamos.

In [10]:
# eliminando os dados desnecessários
df2_sort_clean = df2_sort.dropna()

## 3.0 Previsão das Próximos 6 dias de Faturamento

In [ ]:
# Cópia segura dos dados
df2_sort_clean = df2_sort_clean.copy()

# Instanciando o modelo
modelo_preditivo = pkl.load(open('../Modelos/XGBRegressorFinalVersao1.pkl', 'rb'))

# Realizando a Predição
predicao = modelo_preditivo.predict(df2_sort_clean)

# Armazenando os dados no Dataset
df2_sort_clean.loc[:,'PredictionSalesNextSexWeek'] = np.round(predicao.tolist(), 2)

## 4.0 Salvando os Dados

In [12]:
df2_sort_clean.to_parquet(path='../DadosPredicao/BasePredita.parquet', engine='fastparquet', index=False)